Also we try to determine the optimal volume of the training sample.

- Threshold vas fixed at 0.00075
- We try to hold first kind error about 0.05

**Here are results:**

| Training sample volume | Training time | First Kind Error | Second Kind Error |               Comments                | Conclusion |
|------------------------|---------------|------------------|-------------------|---------------------------------------|------------|
| 20 000 | ~30min | 0.050925 | 0.1667 | First 20 000 rows from first log file | Pretty fine and fast train |
| 877 616 | ~50hours | - | - | First 30 log files  | Too long fitting |
| 571145 |  |  |  | First 30 log files with filtered crawlers  | Оverfitting |
|  |  |  |  | First 30 log files with filtered crawlers  | Оverfitting |

In [1]:
import pandas as pd
import numpy as np

import scipy.sparse
import sklearn.feature_extraction

import matplotlib.pylab as plt
%matplotlib inline

from tqdm import tqdm
import platform

pd.set_option("display.max_rows", 10)
pd.set_option('display.max_columns', 1100)

import os

%pylab inline
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


/home/levabd/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
main_data = np.load('df/main_data.npy').tolist()
values_data = np.load('df/values_data.npy').tolist()
order_data = np.load('df/order_data.npy').tolist()

main_df = pd.DataFrame(main_data)

main_df

,device_class_code,ip,os_code,os_family_code,timestamp,ua_class_code,ua_family_code,ua_string,ua_version
0,desktop,95.181.252.91,windows_7,windows,1485900038,browser,chrome,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,chrome55.0.2883.87
1,smartphone,31.148.3.19,android_4_4,android,1485900079,mobile_browser,chrome_mobile,Mozilla/5.0 (Linux; Android 4.4.2; Zera S Buil...,chrome_mobile49.0.2623.105
2,desktop,188.162.183.66,windows_7,windows,1485900103,browser,firefox,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:51.0) G...,firefox51.0
3,desktop,37.144.52.103,windows_xp,windows,1485900105,browser,ie,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,ie8.0
4,desktop,91.201.178.51,windows_8_1,windows,1485900108,browser,opera,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,opera40.0.2308.81
...,...,...,...,...,...,...,...,...,...
571140,smartphone,89.237.103.208,android_6,android,1488585286,mobile_browser,chrome_mobile,Mozilla/5.0 (Linux; Android 6.0.1; SM-G925F Bu...,chrome_mobile56.0.2924.87
571141,desktop,141.0.12.169,windows_98,windows,1488585377,browser,opera,Mozilla/4.0 (Windows 98; US) Opera 12.16 [en],opera12.16
571142,desktop,131.253.24.147,windows_7,windows,1488585395,browser,ie,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,ie9.0
571143,desktop,176.195.122.100,windows_7,windows,1488585447,browser,ie,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,ie8.0


In [3]:
important_values_keys_set = {
    'Accept', 
    'Accept-Charset', 
    'Accept-Encoding'
}
important_orders_keys_set = {
    'Upgrade-Insecure-Requests',
    'Accept', 
    'If-Modified-Since',
    'Host', 
    'Connection', 
    'User-Agent', 
    'From', 
    'Accept-Encoding' 
}

orders_vectorizer = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)
values_vectorizer = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)

list_ua = pd.DataFrame(main_data).ua_string.value_counts().index.tolist()

# For NaN Useragent
list_ua.append('NaN')

from lib.parsers.logParser import LogParser

l_parser = LogParser(log_folder='Logs/')
l_parser.reassign_orders_values(order_data, values_data)
full_sparce_dummy = l_parser.prepare_data(orders_vectorizer, values_vectorizer, important_orders_keys_set, important_values_keys_set, fit_dict=True)

from sklearn import preprocessing

lb = preprocessing.LabelBinarizer(sparse_output=True)
lb.fit(list_ua)
y = lb.transform(pd.DataFrame(main_data).ua_string.fillna('NaN'))

  7%|▋         | 40843/571145 [00:00<00:01, 378458.18it/s]

Sparse dummy orders shape: 
(571145, 50)


100%|██████████| 571145/571145 [00:01<00:00, 326922.56it/s]


Sparse dummy values shape: 
(571145, 516)


### Train model

In [ ]:
%%time

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

smart_clf = OneVsRestClassifier(LogisticRegression(C=100))
smart_clf.fit(full_sparce_dummy, y)

## Test part

### Prepare data (50/50 bots and human mixed)

In [ ]:
# -- HUMANS --

l_parser = LogParser(log_folder='Logs/')

main_data, values_data, order_data = l_parser.parse_train_sample(45, 46)

# -- BOTS --

l_parser = LogParser(log_folder='Logs/')

main_bot_data = np.load('df/main2_bot_data.npy').tolist()
values_bot_data = np.load('df/values2_bot_data.npy').tolist()
order_bot_data = np.load('df/order2_bot_data.npy').tolist()

main_all = pd.DataFrame(main_data[:20000] + main_bot_data[:20000].to_dict('records'))
main_all['is_human'] = False
main_all.loc[0:19999,'is_human'] = True

values_all = values_data[:20000] + values_bot_data[:20000]
order_all = order_data[:20000] + order_bot_data[:20000]

list_all_ua = main_all.User_Agent.value_counts().index.tolist()

# For NaN Useragent
list_all_ua.append('0')

l_parser.reassign_orders_values(order_all, values_all)

test_sparce_dummy = l_parser.prepare_data(orders_vectorizer, values_vectorizer, important_orders_keys_set, important_values_keys_set, fit_dict=False)

lb_test = preprocessing.LabelBinarizer(sparse_output=True)
lb_test.fit(list_all_ua)
y_test = lb_test.transform(main_all.User_Agent.fillna('0'))

test_sparce_dummy.shape

### Test predictions proba (Threshold 0.024)

In [ ]:
from lib.thresholdPredictions import ThresholdPredictions

pred = ThresholdPredictions(user_agent_list=lb.classes_.tolist(), full_user_agent_list=lb_test.classes_.tolist(), clf=smart_clf)
y_test_names, y_predicted, compare_answers, is_bot, answers_count = pred.bot_predict(test_sparce_dummy, y_test, 0.024072, sparce_y=True)

In [ ]:
compare_frame = pd.concat(
    [
        pd.DataFrame(y_test_names),
        y_predicted, 
        pd.DataFrame(compare_answers),  
        #pd.DataFrame(main_all[['is_human']]), 
        pd.DataFrame(is_bot), 
        pd.DataFrame(answers_count)
    ], keys=['test', 'predicted', 'correctness', 'is_bot_predicted', 'count'], axis=1, join='inner')

    
compare_frame['is_human'] = False
compare_frame.loc[0:19999,'is_human'] = True
compare_frame

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, auc

headers_cm = confusion_matrix(~compare_frame.is_human, compare_frame.is_bot_predicted)

In [ ]:
headers_FP = headers_cm.sum(axis=0) - np.diag(headers_cm)  
headers_FN = headers_cm.sum(axis=1) - np.diag(headers_cm)
headers_TP = np.diag(headers_cm)
headers_TN = headers_cm.sum() - (headers_FP + headers_FN + headers_TP)

In [ ]:
print('TP: {}'.format(headers_TP))
print('TN: {}'.format(headers_TN))
print("FP: {}".format(headers_FP))
print("FN: {}".format(headers_FN))
print("Accuracy (ACC): {}".format((headers_TP + headers_TN) / (headers_TP + headers_TN + headers_FP + headers_FN)))
print("Sensitivity, hit rate, recall, or true positive rate (TPR): {}".format(headers_TP / (headers_TP + headers_FN)))
print("Precision or positive predictive value (PPV): {}".format(headers_TP / (headers_TP + headers_FP)))

In [ ]:
first_kind_error = []
second_kind_error = []
for i, bot_prediction in enumerate(is_bot):
    cur_first_kind_error = False
    cur_second_kind_error = False
    if bot_prediction and i<20000:
        cur_first_kind_error = True
    if not bot_prediction and i>19999:
        cur_second_kind_error = True
    first_kind_error.append(cur_first_kind_error)
    second_kind_error.append(cur_second_kind_error)    

compare_frame['first_kind_error'] = first_kind_error
compare_frame['second_kind_error'] = second_kind_error
compare_frame

In [ ]:
print('Ошибка первого рода (когда мы принимаем нормального пользователя за бота): {}'.format(sum(first_kind_error) / y_test.shape[0]))
print('Ошибка второго рода (когда мы принимаем бота за нормального пользователя): {}'.format(sum(second_kind_error) / y_test.shape[0]))

### Threshold 0.00075

In [61]:
pred = ThresholdPredictions(user_agent_list=lb.classes_.tolist(), full_user_agent_list=lb_test.classes_.tolist(), clf=smart_clf)
y_test_names, y_predicted, compare_answers, is_bot, answers_count = pred.bot_predict(test_sparce_dummy, y_test, 0.00078, sparce_y=True)

40000it [00:03, 10086.17it/s]


In [62]:
compare_frame = pd.concat(
    [
        pd.DataFrame(y_test_names),
        y_predicted, 
        pd.DataFrame(compare_answers),  
        #pd.DataFrame(main_all[['is_human']]), 
        pd.DataFrame(is_bot), 
        pd.DataFrame(answers_count)
    ], keys=['test', 'predicted', 'correctness', 'is_bot_predicted', 'count'], axis=1, join='inner')

    
compare_frame['is_human'] = False
compare_frame.loc[0:19999,'is_human'] = True

headers_cm = confusion_matrix(~compare_frame.is_human, compare_frame.is_bot_predicted)

headers_FP = headers_cm.sum(axis=0) - np.diag(headers_cm)  
headers_FN = headers_cm.sum(axis=1) - np.diag(headers_cm)
headers_TP = np.diag(headers_cm)
headers_TN = headers_cm.sum() - (headers_FP + headers_FN + headers_TP)

print('TP: {}'.format(headers_TP))
print('TN: {}'.format(headers_TN))
print("FP: {}".format(headers_FP))
print("FN: {}".format(headers_FN))
print("Accuracy (ACC): {}".format((headers_TP + headers_TN) / (headers_TP + headers_TN + headers_FP + headers_FN)))
print("Sensitivity, hit rate, recall, or true positive rate (TPR): {}".format(headers_TP / (headers_TP + headers_FN)))
print("Precision or positive predictive value (PPV): {}".format(headers_TP / (headers_TP + headers_FP)))

first_kind_error = []
second_kind_error = []
for i, bot_prediction in enumerate(is_bot):
    cur_first_kind_error = False
    cur_second_kind_error = False
    if bot_prediction and i<20000:
        cur_first_kind_error = True
    if not bot_prediction and i>19999:
        cur_second_kind_error = True
    first_kind_error.append(cur_first_kind_error)
    second_kind_error.append(cur_second_kind_error)    

compare_frame['first_kind_error'] = first_kind_error
compare_frame['second_kind_error'] = second_kind_error

print('Ошибка первого рода (когда мы принимаем нормального пользователя за бота): {}'.format(sum(first_kind_error) / y_test.shape[0]))
print('Ошибка второго рода (когда мы принимаем бота за нормального пользователя): {}'.format(sum(second_kind_error) / y_test.shape[0]))

TP: [17963 13332]
TN: [13332 17963]
FP: [6668 2037]
FN: [2037 6668]
Accuracy (ACC): [ 0.782375  0.782375]
Sensitivity, hit rate, recall, or true positive rate (TPR): [ 0.89815  0.6666 ]
Precision or positive predictive value (PPV): [ 0.72928424  0.86746047]
Ошибка первого рода (когда мы принимаем нормального пользователя за бота): 0.050925
Ошибка второго рода (когда мы принимаем бота за нормального пользователя): 0.1667
